Que3.-
In this dataset you have to predict that whether we have to give ATTORNEY to the clients or not (0=Yes, 1=No) based on below factors.
CLMAGE - Age of Client
CLMSEX - Sex of client (0=Male, 1= Female)
SEATBELT - Whether client has Seatbelt or not (0=Yes, 1=No)
CLMINSUR - Whether Client give Insurance or not (0=Yes, 1=No)
LOSS - Loss that client faced

Prepare model by using Decision Tree, Random Forest , AdaBoost, CatBoost, XGBoost & Calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score.

Dataset : Claimants.csv

### pre-requisites

In [1]:
import warnings
warnings.filterwarnings(action="ignore")

### import required packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### load the data

In [3]:
df = pd.read_csv('claimants.csv')
df.head()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
0,0,0.0,1.0,0.0,50.0,34.940
1,1,1.0,0.0,0.0,18.0,0.891
2,1,0.0,1.0,0.0,5.0,0.330
3,0,0.0,1.0,1.0,31.0,0.037
4,1,0.0,1.0,0.0,30.0,0.038


### EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ATTORNEY  1340 non-null   int64  
 1   CLMSEX    1328 non-null   float64
 2   CLMINSUR  1299 non-null   float64
 3   SEATBELT  1292 non-null   float64
 4   CLMAGE    1151 non-null   float64
 5   LOSS      1340 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 62.9 KB


In [5]:
df.isna().sum()

ATTORNEY      0
CLMSEX       12
CLMINSUR     41
SEATBELT     48
CLMAGE      189
LOSS          0
dtype: int64

In [7]:
df['CLMAGE'] = df['CLMAGE'].fillna(df['CLMAGE'].mean())
l = ['CLMSEX','CLMINSUR','SEATBELT']
for i in l:
    df[i] = df[i].fillna(df[i].mode()[0])

In [8]:
df.isna().sum()

ATTORNEY    0
CLMSEX      0
CLMINSUR    0
SEATBELT    0
CLMAGE      0
LOSS        0
dtype: int64

In [10]:
df['ATTORNEY'].value_counts()

ATTORNEY
0    685
1    655
Name: count, dtype: int64

In [11]:
df.corr()

,ATTORNEY,CLMSEX,CLMINSUR,SEATBELT,CLMAGE,LOSS
ATTORNEY,1.000000,0.076564,0.076631,-0.055848,0.009886,-0.219715
CLMSEX,0.076564,1.000000,0.065976,-0.016327,-0.022444,-0.028412
CLMINSUR,0.076631,0.065976,1.000000,0.019953,0.044706,0.004641
SEATBELT,-0.055848,-0.016327,0.019953,1.000000,-0.025379,0.111879
CLMAGE,0.009886,-0.022444,0.044706,-0.025379,1.000000,0.057641
LOSS,-0.219715,-0.028412,0.004641,0.111879,0.057641,1.000000


### pre-processing

In [12]:
# split data into x and y
x = df[['LOSS']]
y = df['ATTORNEY']

In [13]:
# split the data into train and test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=123456)

### train the models

#### Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

def create_model_decision_tree():
    model = DecisionTreeClassifier(max_depth=50)
    model.fit(x_train, y_train)
    return model    

#### Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

def create_model_random_forest():
    model = RandomForestClassifier(n_estimators=100)
    model.fit(x_train, y_train)
    return model

#### CAT Boost

In [16]:
from catboost import CatBoostClassifier

def create_model_catboost():
    model = CatBoostClassifier(logging_level='Silent')
    model.fit(x_train, y_train)
    return model

#### AdaBoost

In [17]:
from sklearn.ensemble import AdaBoostClassifier

def create_model_ada_boost():
    model = AdaBoostClassifier()
    model.fit(x_train, y_train)
    return model

#### XGBoost

In [18]:
from xgboost import XGBClassifier

def create_model_xgboost():
    model = XGBClassifier()
    model.fit(x_train, y_train)
    return model

### model evaluation

In [19]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model):
    # training accuracry
    y_pred = model.predict(x_train)
    y_true = y_train

    # calculate training accuracy
    training_accuracy = accuracy_score(y_true, y_pred)

    # testing accuracy
    y_pred = model.predict(x_test)
    y_true = y_test

    print(confusion_matrix(y_true, y_pred))

    # get the performance metrics for testing
    testing_accuracy = accuracy_score(y_true, y_pred)
    testing_precision = precision_score(y_true, y_pred,average='micro')
    testing_recall = recall_score(y_true, y_pred,average='micro')
    testing_f1 = f1_score(y_true, y_pred,average='micro')

    return training_accuracy, testing_accuracy, testing_precision, testing_recall, testing_f1

### generate performance chart

In [20]:
models = [
    ("Decision Tree", create_model_decision_tree(), (0, 0)),
    ("Random Forest", create_model_random_forest(), (0, 1)),
    ("CAT Boost", create_model_catboost(), (1, 0)),
    ("Ada Boost", create_model_ada_boost(), (1, 1)),
    ("XGBoost", create_model_xgboost(), (2, 0))
]

In [21]:
performance_data = []
for model_name, model, position in models:
    train_accuracy, test_accuracy, precision, recall, f1 = evaluate_model(model)
    performance_data.append([
        model_name, f"{train_accuracy * 100:0.2f}%", f"{test_accuracy * 100:0.2f}%", precision, recall, f1
    ])
    
performance_chart = pd.DataFrame(performance_data, 
            columns=["Model", "Train Accuracy", "Test Accuracy", "Precision", "Recall", "F1"])
performance_chart

[[135  72]
 [ 73 123]]
[[133  74]
 [ 66 130]]
[[155  52]
 [ 66 130]]
[[155  52]
 [ 68 128]]
[[150  57]
 [ 66 130]]


,Model,Train Accuracy,Test Accuracy,Precision,Recall,F1
0,Decision Tree,90.82%,64.02%,0.640199,0.640199,0.640199
1,Random Forest,90.82%,65.26%,0.652605,0.652605,0.652605
2,CAT Boost,73.85%,70.72%,0.707196,0.707196,0.707196
3,Ada Boost,73.43%,70.22%,0.702233,0.702233,0.702233
4,XGBoost,75.67%,69.48%,0.694789,0.694789,0.694789
